In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Video Processing

In [ ]:
import cv2
import torch
from torchvision.transforms import functional as F
from PIL import Image
import math
import numpy as np

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = torch.load('/content/drive/MyDrive/drone_detection_ssd_model.pth', map_location=torch.device('cpu'))
model.eval().to(device)

def frame_to_tensor(frame):
    img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    img = F.to_tensor(img).to(device)
    return img

def draw_bounding_boxes(frame, boxes, labels):
    for box in boxes:
        x_min, y_min, x_max, y_max = map(int, box)
        cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
    return frame

def process_video(input_path, output_path):
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    codec = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, codec, fps, (width, height))
    print(f"FPS: {fps}")

    idx = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        img_tensor = frame_to_tensor(frame).unsqueeze(0)
        with torch.no_grad():
            outputs = model(img_tensor)

        predicted_box = torch.argmax(outputs[0]['scores'])
        labels = outputs[0]['labels'][predicted_box].cpu().numpy()
        boxes = outputs[0]['boxes'][predicted_box].unsqueeze(0).cpu().numpy()

        x_min, y_min, x_max, y_max = boxes[0]

        center_box = ((x_min + x_max) / 2, (y_min + y_max) / 2)
        delta_x = center_box[0] - width / 2
        delta_y = center_box[1] - height / 2
        angle_rad = math.atan2(delta_y, delta_x)
        angle_deg = math.degrees(angle_rad)
        angle_deg_from_y = (angle_deg + 90) % 360
        if angle_deg_from_y < 0:
          angle_deg_from_y += 360
        print(f"{idx} Angle in degrees from positive y-axis: {angle_deg_from_y}")
        idx += 1

        start_point = (int(width / 2), int(height / 2))
        end_point = (int(center_box[0]), int(center_box[1]))
        color = (255, 0, 0)
        thickness = 2
        image = cv2.line(frame, start_point, end_point, color, thickness)

        frame = draw_bounding_boxes(frame, boxes, labels)
        out.write(frame)

    cap.release()
    out.release()
    print("Processed video saved at:", output_path)

In [ ]:
input_video_path = '/content/drive/MyDrive/drone_short_vid.mp4'
output_video_path = '/content/drive/MyDrive/output_video_with_bboxes.mp4'
process_video(input_video_path, output_video_path)

FPS: 25
0 Angle in degrees from positive y-axis: 183.69313528185134
1 Angle in degrees from positive y-axis: 184.53700753322846
2 Angle in degrees from positive y-axis: 184.451082764353
3 Angle in degrees from positive y-axis: 185.3526793025631
4 Angle in degrees from positive y-axis: 185.13229771252978
5 Angle in degrees from positive y-axis: 185.08717887358662
6 Angle in degrees from positive y-axis: 184.7797504611056
7 Angle in degrees from positive y-axis: 182.42081733639367
8 Angle in degrees from positive y-axis: 188.70748370369364
9 Angle in degrees from positive y-axis: 182.9279575837651
10 Angle in degrees from positive y-axis: 184.05687861380352
11 Angle in degrees from positive y-axis: 183.954786159454
12 Angle in degrees from positive y-axis: 183.35049347828
13 Angle in degrees from positive y-axis: 180.95419442178667
14 Angle in degrees from positive y-axis: 175.95700946890514
15 Angle in degrees from positive y-axis: 175.70029126987544
16 Angle in degrees from positive y-

## Creating omnx

In [ ]:
!pip install onnx --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 45.8 MB/s eta 0:00:00


In [ ]:
import torch
import torchvision.models.detection as detection
import onnx

model = torch.load('/content/drive/MyDrive/drone_detection_ssd_model.pth', map_location=torch.device('cpu'))
model.eval()

dummy_input = torch.randn(1, 3, 320, 320)

onnx_model_path = "/content/drive/MyDrive/ssdlite320_mobilenet_v3_large.onnx"
torch.onnx.export(
    model,
    dummy_input,
    onnx_model_path,
    verbose=True,
    opset_version=11,
    input_names=['input'],
    output_names=['output'],
    dynamic_axes={
        'input': {0: 'batch_size'},
        'output': {0: 'batch_size'}
    }
)

print(f"Model has been converted to ONNX and saved at {onnx_model_path}")

/usr/local/lib/python3.10/dist-packages/torchvision/ops/boxes.py:166: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  boxes_x = torch.min(boxes_x, torch.tensor(width, dtype=boxes.dtype, device=boxes.device))
/usr/local/lib/python3.10/dist-packages/torchvision/ops/boxes.py:168: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  boxes_y = torch.min(boxes_y, torch.tensor(height, dtype=boxes.dtype, device=boxes.device))
/usr/local/lib/python3.10/dist-packages/torchvision/models/detection/transform.py:308: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Model has been converted to ONNX and saved at /content/drive/MyDrive/ssdlite320_mobilenet_v3_large.onnx


## Checking omnx version

In [ ]:
import onnx
onnx_model = onnx.load("/content/drive/MyDrive/ssdlite320_mobilenet_v3_large.onnx")
onnx.checker.check_model(onnx_model)

You can also visualize the omnx on https://netron.app/ by uploading the model to netron

## Omnx runtime

In [ ]:
!pip install onnxruntime --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.1 MB/s eta 0:00:00


In [ ]:
import onnxruntime

onnx_input = onnx_program.adapt_torch_inputs_to_onnx(torch_input)
print(f"Input length: {len(onnx_input)}")
print(f"Sample input: {onnx_input}")

ort_session = onnxruntime.InferenceSession("./my_image_classifier.onnx", providers=['CPUExecutionProvider'])

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

onnxruntime_input = {k.name: to_numpy(v) for k, v in zip(ort_session.get_inputs(), onnx_input)}

onnxruntime_outputs = ort_session.run(None, onnxruntime_input)